In [17]:
import re
import pandas as pd
from pathlib import Path
from datetime import datetime

from SCRIPTS.redditLinkRetriever import fetch_saved_post_links, save_links_txt
from SCRIPTS.mediaDownloader import download_embedded_media
from SCRIPTS.mediaOrganizer import organize_downloads
from SCRIPTS.redgifDownloader import process_external
from SCRIPTS.cloudflareUploader import upload_media

In [18]:
DRY_RUN = False # Set to True to avoid making any changes

In [19]:
# Retrieve saved post links for the specified user
links = fetch_saved_post_links()

In [20]:
links[:5]

['https://www.reddit.com/r/slutsofsnapchat/comments/1ob41f5/cum_play_with_a_slutty_19yr_old_sexting_sextapes',
 'https://www.reddit.com/r/slutsofsnapchat/comments/1ob2xjm/telegram_creamykikii_cumslut_what_i_offer_gfe',
 'https://www.reddit.com/r/ABGHeavens/comments/1ob1yuu/best_abg',
 'https://www.reddit.com/r/HentaiBullying/comments/1o9n30u/they_use_me_to_make_money_and_their_own',
 'https://www.reddit.com/r/collegesluts/comments/1oaqcv4/freshman_with_biggest_tits_in_class']

# NEW POST VALIDATION

This section validates new posts from reddits saved folder

In [21]:
csv_path = Path("ordered_posts.csv")
raw_df = pd.read_csv(csv_path)

POST_ID_RE = re.compile(r"/comments/([a-z0-9]+)(?:[/?#]|$)", re.IGNORECASE)
SHORT_RE   = re.compile(r"redd\.it/([a-z0-9]+)(?:[/?#]|$)", re.IGNORECASE)
max_order_num = raw_df.order_num.max()

def strip_trailing_slash(url: str) -> str:
    # remove trailing slashes only at the very end (doesn't touch scheme)
    return url.rstrip("/")

def extract_post_id(url: str) -> str | None:
    """
    Try to extract a post id from:
      - standard permalink: .../comments/<postid>/...
      - shortlink: https://redd.it/<postid>
    """
    m = POST_ID_RE.search(url)
    if m:
        return m.group(1)
    m = SHORT_RE.search(url)
    if m:
        return m.group(1)
    return None

existing_ids = set(str(x).lower() for x in raw_df.get("post_id", pd.Series([])).dropna())

new_rows = []
next_order = max_order_num + 1
seen_in_batch = set()  # avoid duplicates within this run

for raw_link in reversed(links):
    link = strip_trailing_slash(raw_link)
    post_id = extract_post_id(link)
    if not post_id:
        continue
    pid = post_id.lower()

    # Only add if NOT already in CSV and not already queued this batch
    if pid in existing_ids or pid in seen_in_batch:
        continue

    new_rows.append({
        "order_num": next_order,
        "link": link,
        "post_id": post_id,
        "date_added": datetime.utcnow().isoformat(timespec="seconds"),
    })
    seen_in_batch.add(pid)
    next_order += 1

# Preview as a DataFrame
new_df = pd.DataFrame(new_rows)
new_df


C:\Users\minds\AppData\Local\Temp\ipykernel_13668\787629229.py:47: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "date_added": datetime.utcnow().isoformat(timespec="seconds"),


,order_num,link,post_id,date_added
0,1286,https://www.reddit.com/r/bangmybully/comments/...,1o8ba01,2025-10-20T01:05:38
1,1287,https://www.reddit.com/r/ABGHeavens/comments/1...,1o97c82,2025-10-20T01:05:38
2,1288,https://www.reddit.com/r/bangmybully/comments/...,1o8bta5,2025-10-20T01:05:38
3,1289,https://www.reddit.com/r/ABGHeavens/comments/1...,1oa6x2t,2025-10-20T01:05:38
4,1290,https://www.reddit.com/r/cumsluts/comments/1o9...,1o9yw0m,2025-10-20T01:05:38
5,1291,https://www.reddit.com/r/bangmybully/comments/...,1o9tk5e,2025-10-20T01:05:38
6,1292,https://www.reddit.com/r/biosuits/comments/1o9...,1o9eszy,2025-10-20T01:05:38
7,1293,https://www.reddit.com/r/SluttyConfessions/com...,1o9oow5,2025-10-20T01:05:38
8,1294,https://www.reddit.com/r/slutsofsnapchat/comme...,1o9t13k,2025-10-20T01:05:38
9,1295,https://www.reddit.com/r/Rapekink/comments/1o9...,1o9fjdk,2025-10-20T01:05:38


In [22]:
import SCRIPTS.jsonDownloader as jd
import importlib
importlib.reload(jd)

jd.configure(
    DATA_ROOT="Out",
    SKIP_EXISTING=False,
    REPORTS_DIR="Out/__reports"
    )

summary = jd.process_all(new_df["link"].tolist(), show_progress=True)
summary

  0%|          | 0/26 [00:00<?, ?post/s]

Done. Success: 26, Skipped: 0, Failed: 0


{'success': 26, 'skipped': 0, 'failed': 0}

# MEDIA DOWNLOADER
Reviews the external and media json folders in **Out/**, downloading:
- Images
- Gifs
- Videos

In [23]:
folders = ["external", "media"]
download_stats = []

# point to your inputs/outputs explicitly
for mediaType in folders:
    download_stats.append(download_embedded_media(
        media_json_dir=Path("Out/" + mediaType),   # where your *.json live
        media_out_dir=Path("Media/media_files"),  # where downloads should go
        write_fail_csv_to=Path("Media/__reports/media_report" + datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"),
        show_progress=True,
    ))

download_stats

[{'downloaded': 0,
  'failed': 14,
  'skipped': 0,
  'fail_rows': [{'id': '1o85il3', 'reason': 'no_reddit_media_url'},
   {'id': '1o8ba01', 'reason': 'no_reddit_media_url'},
   {'id': '1o8bta5', 'reason': 'no_reddit_media_url'},
   {'id': '1o8r5ic', 'reason': 'no_reddit_media_url'},
   {'id': '1o9t13k', 'reason': 'no_reddit_media_url'},
   {'id': '1o9tk5e', 'reason': 'no_reddit_media_url'},
   {'id': '1o9yw0m', 'reason': 'no_reddit_media_url'},
   {'id': '1oa631j', 'reason': 'no_reddit_media_url'},
   {'id': '1oa6chl', 'reason': 'no_reddit_media_url'},
   {'id': '1oa90lu', 'reason': 'no_reddit_media_url'},
   {'id': '1oabnlx', 'reason': 'no_reddit_media_url'},
   {'id': '1oaqcv4', 'reason': 'no_reddit_media_url'},
   {'id': '1oaw7d3', 'reason': 'no_reddit_media_url'},
   {'id': '1ob2xjm', 'reason': 'no_reddit_media_url'}],
  'out_dir': WindowsPath('Media/media_files'),
  'json_dir': WindowsPath('Out/external')},
 {'downloaded': 57,
  'failed': 0,
  'skipped': 0,
  'fail_rows': [],
  'o

In [24]:
move_stats = organize_downloads(
    input_dir="Media/media_files",  # where your downloader wrote files
    output_dir="Media",             # where Images/, Videos/, Gifs/ live
    strategy="move",
    conflict="rename",
    show_progress=True,
    dry_run=False,                  # set True to preview
    prune_empty_galleries=True,     # remove empty src folders after moving
)

move_stats

Organizing media:   0%|          | 0/57 [00:00<?, ?file/s]

{'moved': 57,
 'copied': 0,
 'linked': 0,
 'skipped': 0,
 'unknown': 0,
 'dry_run': False,
 'strategy': 'move',
 'conflict': 'rename',
 'input_dir': 'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\media_files',
 'output_dir': 'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media',
 'errors': [],
 'created_dirs': {'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\Gifs\\1ob41f5',
  'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\Images',
  'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\Images\\1o97c82',
  'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\Images\\1o9n30u',
  'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\Images\\1oa6s16',
  'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\Images\\1oa6x2t',
  'S:\\minds\\Desktop\\Downloader and Reddit System\\Saved-Reddit\\Media\\Images\\1oaee68',
  'S:\\minds\

# REDGIF DOWNLOADER

Downloads redgifs from external json folder in **Out/**

In [25]:
stats = process_external(
    media_json_dir=Path("Out/external"),
    media_out_dir=Path("Media/RedGiphys"),
    write_fail_csv_to=Path("Media/__reports/redgif_report_" + datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"),
    write_links_csv_to=Path("Media/__reports/external_links.csv"),
    show_progress=True,
    dry_run=False,
    overwrite_downloads=False,
)

stats

Found 14 external post JSONs in Out\external


Scanning external posts:   0%|          | 0/14 [00:00<?, ?post/s]

[REDGIFS] id=1o85il3 -> 1o85il3.mp4
[REDGIFS] id=1o8ba01 -> 1o8ba01.mp4
[REDGIFS] id=1o8bta5 -> 1o8bta5.mp4
[REDGIFS] id=1o8r5ic -> 1o8r5ic.mp4
[REDGIFS] id=1o9t13k -> 1o9t13k.mp4
[REDGIFS] id=1o9tk5e -> 1o9tk5e.mp4
[REDGIFS] id=1o9yw0m -> 1o9yw0m.mp4
[REDGIFS] id=1oa631j -> 1oa631j.mp4
[REDGIFS] id=1oa6chl -> 1oa6chl.mp4
[REDGIFS] id=1oa90lu -> 1oa90lu.mp4
[REDGIFS] id=1oabnlx -> 1oabnlx.mp4
[REDGIFS] id=1oaqcv4 -> 1oaqcv4.mp4
[REDGIFS] id=1oaw7d3 -> 1oaw7d3.mp4
[REDGIFS] id=1ob2xjm -> 1ob2xjm.mp4
Saved external links to: S:\minds\Desktop\Downloader and Reddit System\Saved-Reddit\Media\__reports\external_links.csv


{'external_rows': [{'id': '1o85il3',
   'link': 'https://www.redgifs.com/watch/sandybrownthosesealion',
   'domain': 'www.redgifs.com'},
  {'id': '1o8ba01',
   'link': 'https://www.redgifs.com/watch/barespitefuljavalina',
   'domain': 'www.redgifs.com'},
  {'id': '1o8bta5',
   'link': 'https://www.redgifs.com/watch/antiquewhitenuttyanemone',
   'domain': 'www.redgifs.com'},
  {'id': '1o8r5ic',
   'link': 'https://redgifs.com/watch/grossambitiouscreature',
   'domain': 'redgifs.com'},
  {'id': '1o9t13k',
   'link': 'https://www.redgifs.com/watch/impressivelinenbasil',
   'domain': 'www.redgifs.com'},
  {'id': '1o9tk5e',
   'link': 'https://www.redgifs.com/watch/greenmedicalqueenbee',
   'domain': 'www.redgifs.com'},
  {'id': '1o9yw0m',
   'link': 'https://v3.redgifs.com/watch/hardtofindlightskyblueelkhound',
   'domain': 'v3.redgifs.com'},
  {'id': '1oa631j',
   'link': 'https://v3.redgifs.com/watch/criminalreliablegermanshepherd#23339',
   'domain': 'v3.redgifs.com'},
  {'id': '1oa6chl

# CLOUDFLARE VERIFICATION & UPLOAD

In [29]:
raw_output = []
uploadsData = []

for mediaType in ["Images", "Videos", "Gifs", "RedGiphys"]:
    try:
        result = upload_media(
            input_path=Path("Media/" + mediaType + "/"),  # folder with your media or galleries
            r2_prefix=mediaType,                # one of: Gifs, Images, RedGiphys, Videos
            dry_run=True,                      # True = preview only, False = actually upload
            overwrite=False                    # only overwrite existing files if True
        )

        raw_output.append(result)
        uploadsData.extend(result["planned"])
        
    except Exception as e:
        continue

In [36]:
results_df = pd.DataFrame(uploadsData)
pd.set_option('display.max_rows', None)
results_df

,local,r2_key,bytes,content_type
0,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o9eszy.png,7138882,image/png
1,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o97c82/01.jpg,121746,image/jpeg
2,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o97c82/02.jpg,102415,image/jpeg
3,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o97c82/03.jpg,69554,image/jpeg
4,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o97c82/04.jpg,190367,image/jpeg
5,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o97c82/05.jpg,124637,image/jpeg
6,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o9n30u/01.jpg,59141,image/jpeg
7,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o9n30u/02.jpg,137872,image/jpeg
8,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o9n30u/03.jpg,73782,image/jpeg
9,S:\minds\Desktop\Downloader and Reddit System\...,Images/1o9n30u/04.jpg,82437,image/jpeg
